In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType
from pyspark.sql.functions import explode, expr, from_json, col

In [29]:
spark = (SparkSession
         .builder
         .config("spark.streaming.stopGracefullyOnShutdown", True)
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
         .config("spark.sql.shuffle.partitions", 4)
         .master("local[*]")
         .appName("Reading from file").getOrCreate())

In [30]:
spark.conf.set("spark.sql.streaming.schemaInterface", True)
spark

In [31]:
kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "spark_kafka-kafka:29092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

In [33]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [34]:
kafka_df_json = kafka_df.withColumn("value", expr("cast(value as string)"))
# kafka_df_json.show()

In [35]:
devices_element_schema = StructType([
    StructField('deviceId', StringType(), True),
    StructField('measure', StringType(), True),
    StructField('status', StringType(), True),
    StructField('temperature', LongType(), True)
])

devices_schema = ArrayType(devices_element_schema, True)

data_schema = StructType([
    StructField('devices', devices_schema, True)
])

json_schema = StructType([
    StructField('customerId', StringType(), True),
    StructField('data', data_schema, True),
    StructField('eventId', StringType(), True),
    StructField('eventOffset', LongType(), True),
    StructField('eventPublisher', StringType(), True),
    StructField('eventTime', StringType(), True),
])

In [36]:
streaming_df = kafka_df_json.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [37]:
streaming_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)



In [39]:
exploded_df = streaming_df.withColumn('devices', explode(streaming_df.data.devices)).drop('data')

In [40]:
exploded_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [42]:
flattened_df = exploded_df.withColumns({
    'device_id': exploded_df.devices.deviceId,
    'measure' : exploded_df.devices.measure,
    'status': exploded_df.devices.status,
    'temperature' : exploded_df.devices.temperature
}).drop('devices')

In [43]:
flattened_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)



In [ ]:
(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
  .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())

In [ ]:
# (flattened_df
#  .writeStream
#  .format("csv")
#  .outputMode("append")
#  .option("path", "../data/output/events/")        
#  .option("checkpointLocation", "checkpoint_dir")
#  .start()
#  .awaitTermination())